## **Step 1 : Import Libraries**

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.models import Sequential
from keras import layers
from keras.utils import image_dataset_from_directory
from keras.applications import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
DATASET_PATH = "RealWaste"
IMG_SIZE = (524, 524)
BATCH_SIZE = 32
SEED = 42

## **Step 2. Create Train and Validation Dataset**

In [3]:
train_ds = image_dataset_from_directory(
    DATASET_PATH,
    validation_split = 0.2,
    subset = 'training', 
    seed = SEED,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE
)

val_ds = image_dataset_from_directory(
    DATASET_PATH,
    validation_split = 0.2,
    subset = 'validation',
    seed = SEED,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE
)

Found 4752 files belonging to 9 classes.
Using 3802 files for training.
Found 4752 files belonging to 9 classes.
Using 950 files for validation.


## **Step 3. Test and Validation data are Same**

## **Step 4. Name and Count Classes**

In [4]:
class_names = train_ds.class_names
num_classes = len(class_names)

print(f"Class Names are: {class_names}")
print(f"Total Number of classes are: {num_classes}")

Class Names are: ['Cardboard', 'Food Organics', 'Glass', 'Metal', 'Miscellaneous Trash', 'Paper', 'Plastic', 'Textile Trash', 'Vegetation']
Total Number of classes are: 9


## **Step 5. Performance Optimization**

In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.shuffle(1000).prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.shuffle(1000).prefetch(buffer_size = AUTOTUNE)

## **Step 6. Normalization and Augmentation**

In [6]:
normalization = layers.Rescaling(1./255)

data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2)
])

## **Step 7. Building the CNN Model**

In [7]:
base_model = MobileNetV2(
    input_shape = IMG_SIZE + (3,),
    include_top = False,
    weights = 'imagenet'
)

base_model.trainable = False

model = Sequential([
    layers.Input(shape = IMG_SIZE+(3,)),
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation = "relu"),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation = "softmax")
])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9436\2401494480.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step



## **Step 8. Build and Compile the Model**

In [8]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    loss = "sparse_categorical_crossentropy",
    metrics = ['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 474s 4s/step - accuracy: 0.5934 - loss: 1.2012 - val_accuracy: 0.6653 - val_loss: 0.9245
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 637s 5s/step - accuracy: 0.7409 - loss: 0.7379 - val_accuracy: 0.7579 - val_loss: 0.6952
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 623s 5s/step - accuracy: 0.7959 - loss: 0.5915 - val_accuracy: 0.7632 - val_loss: 0.6522
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 623s 5s/step - accuracy: 0.8183 - loss: 0.5186 - val_accuracy: 0.7684 - val_loss: 0.6220
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 606s 5s/step - accuracy: 0.8509 - loss: 0.4464 - val_accuracy: 0.8158 - val_loss: 0.5115
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 537s 4s/step - accuracy: 0.8614 - loss: 0.4166 - val_accuracy: 0.8011 - val_loss: 0.5500
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 544s 4s/step - accuracy: 0.8772 - loss: 0.3648 - val_accuracy: 0.7863 - val_loss: 0.5804
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 467s 4s/step - accuracy: 0.8795 - loss: 0.3465 - val_accu

In [9]:
model.save("ecovision_ai_model.keras")

In [10]:
model.save("ecovision_ai_model.h5")